In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
list=[[  0,   'a',        1],
      [1,   'a',        1],
      [2,   'a',        1],
      [3,   'a',        1]]

df=spark.createDataFrame(list,['day','user','raw_score'])

df.show()

+---+----+---------+
day|user|raw_score|
+---+----+---------+
 0| a| 1|
 1| a| 1|
 2| a| 1|
 3| a| 1|
+---+----+---------+

In [3]:
list=[[  0,   'a',        4],
      [1,   'a',        1],
      [2,   'a',        2],
      [3,   'a',        0]]

df=spark.createDataFrame(list,['day','user','raw_score'])

df.show()

+---+----+---------+
day|user|raw_score|
+---+----+---------+
 0| a| 4|
 1| a| 1|
 2| a| 2|
 3| a| 0|
+---+----+---------+

In [4]:
from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("score", F.expr("""transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                             ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw))"""))\
   .withColumn("zip", F.explode(F.arrays_zip("day","raw_score","score")))\
   .select("user", "zip.*")\
   .show(truncate=False)


+----+---+---------+-----+
user|day|raw_score|score|
+----+---+---------+-----+
a |0 |4 |4.0 |
a |1 |1 |4.6 |
a |2 |2 |6.14 |
a |3 |0 |5.526|
+----+---+---------+-----+

In [5]:
w=Window().partitionBy("user").orderBy("day")
df.withColumn("raw_score", F.when(F.row_number().over(w)==1, F.lit(0)).otherwise(F.col("raw_score")))\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> array(x,i))"""))\
   .withColumn("score", F.col("raw_score1"))\
   .withColumn("score", F.expr("""transform(raw_score1, x-> aggregate(score,cast(0.9 as double)\
                                                            ,(acc,y)->IF(x[1]<=y[1],(acc*x[0])+x[0],0)))"""))\
   .show(truncate=False)

+----+------------+--------------------------------+--------------------+
user|raw_score |raw_score1 |score |
+----+------------+--------------------------------+--------------------+
a |[0, 1, 1, 1]|[[0, 0], [1, 1], [1, 2], [1, 3]]|[0.0, 3.0, 2.0, 1.0]|
+----+------------+--------------------------------+--------------------+

In [6]:
df.show() #sample dataframe
#+---+----+---------+
#|day|user|raw_score|
#+---+----+---------+
#|  0|   a|        1|
#|  1|   a|        1|
#|  2|   a|        1|
#|  3|   a|        1|
#+---+----+---------+


from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("todays_score", F.expr("""transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                             ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw))"""))\
   .withColumn("zip", F.explode(F.arrays_zip("day","raw_score","todays_score")))\
   .select("user", "zip.*")\
   .show(truncate=False)


#+----+---+---------+------------+
#|user|day|raw_score|todays_score|
#+----+---+---------+------------+
#|a   |0  |1        |1.0         |
#|a   |1  |1        |1.9         |
#|a   |2  |1        |2.71        |
#|a   |3  |1        |3.439       |
#+----+---+---------+------------+

+---+----+---------+
day|user|raw_score|
+---+----+---------+
 0| a| 1|
 1| a| 1|
 2| a| 1|
 3| a| 1|
+---+----+---------+

+----+---+---------+------------+
user|day|raw_score|todays_score|
+----+---+---------+------------+
a |0 |1 |1.0 |
a |1 |1 |1.9 |
a |2 |1 |2.71 |
a |3 |1 |3.439 |
+----+---+---------+------------+

In [7]:
from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("score", F.expr("""transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                             ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw))"""))\
   .show(truncate=False)


+----+------------+------------+--------------------------------+-----------------------+
user|raw_score |day |raw_score1 |score |
+----+------------+------------+--------------------------------+-----------------------+
a |[1, 1, 1, 1]|[0, 1, 2, 3]|[[1, 0], [1, 1], [1, 2], [1, 3]]|[1.0, 1.9, 2.71, 3.439]|
+----+------------+------------+--------------------------------+-----------------------+

In [8]:

from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("raw_score2", F.col("raw_score1"))\
   .withColumn("score", F.expr("""transform(raw_score2, x-> aggregate(raw_score1,cast(0 as double)\
                                                 ,(acc,y)->IF(x.index>y.index,0,((acc*0.9)+y.raw))))"""))\
   .show(truncate=False)


+----+------------+------------+--------------------------------+--------------------------------+-----------------------+
user|raw_score |day |raw_score1 |raw_score2 |score |
+----+------------+------------+--------------------------------+--------------------------------+-----------------------+
a |[1, 2, 3, 4]|[0, 1, 2, 3]|[[1, 0], [2, 1], [3, 2], [4, 3]]|[[1, 0], [2, 1], [3, 2], [4, 3]]|[9.049, 8.32, 6.7, 4.0]|
+----+------------+------------+--------------------------------+--------------------------------+-----------------------+

In [9]:

from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("score", F.expr("""sort_array(transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                             ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw)))"""))\
   .withColumn("zip", F.explode(F.arrays_zip("day","raw_score","score")))\
   .select("user", "zip.*")\
   .show(truncate=False)



+----+---+---------+-----------------+
user|day|raw_score|score |
+----+---+---------+-----------------+
a |0 |1 |1.0 |
a |1 |2 |2.9 |
a |2 |3 |5.609999999999999|
a |3 |4 |9.049 |
+----+---+---------+-----------------+

In [10]:

from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("raw_score2", F.col("raw_score1"))\
   .withColumn("score", F.expr("""transform(raw_score2, x-> aggregate(filter(raw_score1,z-> z[0]<=x
   
   
   
   ,cast(0 as double)\
                                                 ,(acc,y)->IF(x.index>y.index,0,((acc*0.9)+y.raw))))"""))\
   .show(truncate=False)


In [11]:

from pyspark.sql import functions as F

df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("raw_score2", F.col("raw_score1"))\
   .withColumn("score", F.expr("""filter(raw_score1,z-> z.index<=1)""")).show()

+----+------------+------------+--------------------+--------------------+----------------+
user| raw_score| day| raw_score1| raw_score2| score|
+----+------------+------------+--------------------+--------------------+----------------+
 a|[1, 2, 3, 4]|[0, 1, 2, 3]|[[1, 0], [2, 1], ...|[[1, 0], [2, 1], ...|[[1, 0], [2, 1]]|
+----+------------+------------+--------------------+--------------------+----------------+

In [12]:
df.groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"))\
  .withColumn("score", F.expr("""transform(raw_score,(x,i)-> array(x,i))""")).show(truncate=False)

+----+------------+--------------------------------+
user|raw_score |score |
+----+------------+--------------------------------+
a |[1, 1, 1, 1]|[[1, 0], [1, 1], [1, 2], [1, 3]]|
+----+------------+--------------------------------+

In [13]:
+---+----+---------+------------+
|day|user|raw_score|todays_score| # Here's the math:
+---+----+---------+------------+
|  0|   a|        1|         1.0| (0 * .90) + 1
|  1|   a|        1|         1.9| (1.0 * .90) + 1
|  2|   a|        1|        2.71| (1.9 * .90) + 1
|  3|   a|        1|       3.439| (2.71 * .90) + 1
+---+----+---------+------------+

In [14]:
list=[[2003],
      [2004],
      [2014],
      [2015],
      [2008],
      [1995]]

df=spark.createDataFrame(list,['year'])
df.show()

+----+
year|
+----+
2003|
2004|
2014|
2015|
2008|
1995|
+----+

In [15]:
list=[['2003-04-16 00:00:00',2003],
            ['2004-05-28 00:00:00',2004],
            ['2014-11-26 00:00:00',2014],
            ['2015-02-06 00:00:00',2015],
            ['2008-06-27 00:00:00',2008]]

df=spark.createDataFrame(list,['movie_theatrical_release_date_upd','year'])

df.show()

+---------------------------------+----+
movie_theatrical_release_date_upd|year|
+---------------------------------+----+
 2003-04-16 00:00:00|2003|
 2004-05-28 00:00:00|2004|
 2014-11-26 00:00:00|2014|
 2015-02-06 00:00:00|2015|
 2008-06-27 00:00:00|2008|
+---------------------------------+----+

In [16]:
from pyspark.sql import functions as F
df.withColumn("decade", F.expr("""int(year-right(year,1))""")).show()

+----+------+
year|decade|
+----+------+
2003| 2000|
2004| 2000|
2014| 2010|
2015| 2010|
2008| 2000|
1995| 1990|
+----+------+

In [17]:
from pyspark.sql import functions as F
df.withColumn("decade", F.regexp_replace("year",'\d(?!.*\d)','0')).show()

+----+------+
year|decade|
+----+------+
2003| 2000|
2004| 2000|
2014| 2010|
2015| 2010|
2008| 2000|
1995| 1990|
+----+------+

In [18]:
from pyspark.sql import functions as F
df.withColumn("decade", F.expr("""concat(substring(year,1,length(year)-1),0)""")).show()

+----+------+
year|decade|
+----+------+
2003| 2000|
2004| 2000|
2014| 2010|
2015| 2010|
2008| 2000|
1995| 1990|
+----+------+

In [19]:
df.withColumn("decade", F.floor(F.col("year")/10)).show()

+----+------+
year|decade|
+----+------+
2003| 200|
2004| 200|
2014| 201|
2015| 201|
2008| 200|
1995| 199|
+----+------+

In [20]:
from pyspark.sql import functions as F
df.withColumn("decade", (F.floor(F.col("year")/10)*10).cast("int")).show()

#+----+------+
#|year|decade|
#+----+------+
#|2003|  2000|
#|2004|  2000|
#|2014|  2010|
#|2015|  2010|
#|2008|  2000|
#+----+------+

df.withColumn("decade", F.to_date(""))

+----+------+
year|decade|
+----+------+
2003| 2000|
2004| 2000|
2014| 2010|
2015| 2010|
2008| 2000|
1995| 1990|
+----+------+

In [21]:
list=[[ '2018-08-01 06:01:00'  ,    1],
      ['2018-08-01 06:01:30'  ,    1],
      ['2018-08-01 09:00:00'  ,    1],
      ['2018-08-01 09:00:00'  ,    2],
      ['2018-08-01 10:15:43'  ,    2],
      ['2018-08-01 11:00:01'  ,    3],
      ['2018-08-01 06:00:13'  ,    4],
      ['2018-08-01 13:00:00'  ,    4],
      ['2018-08-13 14:00:00'  ,    5],
      ['2018-08-13 14:15:03'  ,    5],
      ['2018-08-13 14:45:08'  ,    5],
      ['2018-08-13 14:50:00'  ,    5]]

df=spark.createDataFrame(list,['timestamp','ship'])

df.show()


+-------------------+----+
 timestamp|ship|
+-------------------+----+
2018-08-01 06:01:00| 1|
2018-08-01 06:01:30| 1|
2018-08-01 09:00:00| 1|
2018-08-01 09:00:00| 2|
2018-08-01 10:15:43| 2|
2018-08-01 11:00:01| 3|
2018-08-01 06:00:13| 4|
2018-08-01 13:00:00| 4|
2018-08-13 14:00:00| 5|
2018-08-13 14:15:03| 5|
2018-08-13 14:45:08| 5|
2018-08-13 14:50:00| 5|
+-------------------+----+

In [22]:
list=[[ '2018-08-01 06:01:00'  ,  '2018-08-01 06:01:00' ,1],
      ['2018-08-01 06:01:30'  ,   '2018-08-01 06:01:00' ,2]]

df=spark.createDataFrame(list,['timestamp','timestamp2','number'])

df.show()

+-------------------+-------------------+------+
 timestamp| timestamp2|number|
+-------------------+-------------------+------+
2018-08-01 06:01:00|2018-08-01 06:01:00| 1|
2018-08-01 06:01:30|2018-08-01 06:01:00| 2|
+-------------------+-------------------+------+

In [24]:
cols=['timestamp','timestamp2']
df=df.select(*[F.to_timestamp(x).cast('long').alias(x) for x in df.columns])

df.select(*[x for x in dt.columns if x not in cols],*[F.from_unixtime(x).alias(x) for x in cols]).show()

+------+-------------------+-------------------+
number| timestamp| timestamp2|
+------+-------------------+-------------------+
 1|2018-08-01 06:01:00|2018-08-01 06:01:00|
 2|2018-08-01 06:01:30|2018-08-01 06:01:00|
+------+-------------------+-------------------+

In [25]:
df.withColumn("struct", F.struct(*[F.struct(F.from_unixtime(x)).alias(x) for x in cols]))\
  .select(*[F.col("struct.{}.col1".format(x)).alias(x) for x in cols]).show()

+-------------------+-------------------+
 timestamp| timestamp2|
+-------------------+-------------------+
2018-08-01 06:01:00|2018-08-01 06:01:00|
2018-08-01 06:01:30|2018-08-01 06:01:00|
+-------------------+-------------------+

In [26]:
from functools import reduce
from pyspark.sql.functions import *
new_df = (reduce(
lambda df, cols: df.withColumn(cols, from_unixtime(col(cols), "yyyy-MM-dd HH:mm:ss")),
df.columns,
df))

In [27]:
new_df.show()

+-------------------+-------------------+-------------------+
 timestamp| timestamp2| number|
+-------------------+-------------------+-------------------+
2018-08-01 06:01:00|2018-08-01 06:01:00|1970-01-01 00:00:01|
2018-08-01 06:01:30|2018-08-01 06:01:00|1970-01-01 00:00:02|
+-------------------+-------------------+-------------------+

In [28]:
df.select(*[x for x in df.columns if x not in cols],*[F.from_unixtime(x).alias(x) for x in cols])

Out[10]: DataFrame[number: bigint, timestamp: string, timestamp2: string]

In [29]:
 df=     dt.withColumn('PL_start', from_unixtime('PL_start', "yyyy-MM-dd HH:mm:ss")) \
            .withColumn('PL_end??'  , from_unixtime('PL_end', "yyyy-MM-dd HH:mm:ss"))   \
            .withColumn('MU_start', from_unixtime('MU_start', "yyyy-MM-dd HH:mm:ss")) \
            .withColumn('MU_end'  , from_unixtime('MU_end', "yyyy-MM-dd HH:mm:ss"))   \
            .withColumn('PU_start', from_unixtime('PU_start', "yyyy-MM-dd HH:mm:ss")) \
            .withColumn('PU_end'  , from_unixtime('PU_end', "yyyy-MM-dd HH:mm:ss"))   \
            .withColumn('RE_start', from_unixtime('RE_start', "yyyy-MM-dd HH:mm:ss")) \
            .withColumn('RE_end'  , from_unixtime('RE_end', "yyyy-MM-dd HH:mm:ss"))   \

In [30]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("ship").orderBy(F.unix_timestamp("timestamp")).rangeBetween(-7199, Window.currentRow)
w2=Window().partitionBy("ship").orderBy("timestamp")
w3=Window().orderBy("ship","timestamp")

df.withColumn("trip", F.sum(F.when(F.row_number().over(w2)==1, F.lit(1))\
                       .when(F.size(F.collect_list("ship").over(w1))==1, F.lit(1))\
                       .otherwise(F.lit(0))).over(w3)).orderBy("ship","timestamp").show()

#+-------------------+----+----+
#|          timestamp|ship|trip|
#+-------------------+----+----+
#|2018-08-01 06:01:00|   1|   1|
#|2018-08-01 06:01:30|   1|   1|
#|2018-08-01 09:00:00|   1|   2|
#|2018-08-01 09:00:00|   2|   3|
#|2018-08-01 10:15:43|   2|   3|
#|2018-08-01 11:00:01|   3|   4|
#|2018-08-01 06:00:13|   4|   5|
#|2018-08-01 13:00:00|   4|   6|
#|2018-08-13 14:00:00|   5|   7|
#|2018-08-13 14:15:03|   5|   7|
#|2018-08-13 14:45:08|   5|   7|
#|2018-08-13 14:50:00|   5|   7|
#+-------------------+----+----+

+-------------------+----+----+
 timestamp|ship|trip|
+-------------------+----+----+
2018-08-01 06:01:00| 1| 1|
2018-08-01 06:01:30| 1| 1|
2018-08-01 09:00:00| 1| 2|
2018-08-01 09:00:00| 2| 3|
2018-08-01 10:15:43| 2| 3|
2018-08-01 11:00:01| 3| 4|
2018-08-01 06:00:13| 4| 5|
2018-08-01 13:00:00| 4| 6|
2018-08-13 14:00:00| 5| 7|
2018-08-13 14:15:03| 5| 7|
2018-08-13 14:45:08| 5| 7|
2018-08-13 14:50:00| 5| 7|
+-------------------+----+----+

In [31]:
+----------------------+------+-------+
|        timestamp     | ship | trip  |
+----------------------+------+-------+
| 2018-08-01 06:01:00  |    1 |    1  | # start new ship number
| 2018-08-01 06:01:30  |    1 |    1  | # still within 2 hours of same ship number
| 2018-08-01 09:00:00  |    1 |    2  | # more than 2 hours of same ship number = new trip
| 2018-08-01 09:00:00  |    2 |    3  | # new ship number = new trip
| 2018-08-01 10:15:43  |    2 |    3  | # still within 2 hours of same ship number
| 2018-08-01 11:00:01  |    3 |    4  | # new ship number = new trip
| 2018-08-01 06:00:13  |    4 |    5  | # new ship number = new trip
| 2018-08-01 13:00:00  |    4 |    6  | # more than 2 hours of same ship number = new trip
| 2018-08-13 14:00:00  |    5 |    7  | # new ship number = new trip
| 2018-08-13 14:15:03  |    5 |    7  | # still within 2 hours of same ship number
| 2018-08-13 14:45:08  |    5 |    7  | # still within 2 hours of same ship number
| 2018-08-13 14:50:00  |    5 |    7  | # still within 2 hours of same ship number
+-----------------------------+-------+